In [1]:
import numpy as np
from collections import Counter
from sklearn.preprocessing import LabelEncoder
# 匯入相關所需的模組
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from pathlib import PurePath
import cv2


from sklearn.model_selection import train_test_split
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,Input
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical


In [2]:


number_images_train = np.load('./dataset/images_train.npy')
number_labels_train = np.load('./dataset/labels_train.npy')[:,0]
number_images_test = np.load('./dataset/images_test.npy')
number_labels_test = np.load('./dataset/labels_test.npy')[:,0]
print(Counter(number_labels_train))
print(Counter(number_labels_test))

le = LabelEncoder()
le.fit(number_labels_train)
number_labels_train = le.transform(number_labels_train)
number_labels_test = le.transform(number_labels_test)

Counter({'Q': 373, '6': 372, 'T': 372, '5': 371, 'J': 371, 'A': 371, '8': 370, '2': 369, '4': 368, 'K': 366, '3': 363, '7': 362, '9': 356})
Counter({'9': 102, '7': 97, '3': 97, 'K': 94, '2': 93, '4': 93, '8': 92, '6': 90, 'J': 89, 'T': 88, 'A': 88, '5': 88, 'Q': 85})


In [3]:
# 建立卷積神經網絡模型
numbermodel = Sequential()
numbermodel.add(Input(shape=(70, 70, 1)))
numbermodel.add(Conv2D(32, (3, 3),padding = 'same', activation='relu'))
numbermodel.add(MaxPooling2D((2, 2)))
numbermodel.add(Dropout(0.1))
numbermodel.add(Conv2D(64, (3, 3),padding = 'same', activation='relu'))
numbermodel.add(MaxPooling2D((2, 2)))
numbermodel.add(Dropout(0.1))
numbermodel.add(Conv2D(64, (3, 3),padding = 'same', activation='relu'))
numbermodel.add(MaxPooling2D((2, 2)))
numbermodel.add(Dropout(0.1))
numbermodel.add(Flatten())
numbermodel.add(Dense(128, activation='relu'))
numbermodel.add(Dense(13, activation='softmax'))

# 編譯模型
opt = keras.optimizers.Adam()
numbermodel.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

numbermodel.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 70, 70, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 35, 35, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 35, 35, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 35, 35, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 17, 17, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 13)             │         1,677 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 581,837 (2.22 MB)

 Trainable params: 581,837 (2.22 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
numbermodel.fit(number_images_train, number_labels_train,epochs=80, validation_data=(number_images_test, number_labels_test),batch_size=32)

Epoch 1/80
